<a href="https://colab.research.google.com/github/pedro-henrique-maia/mba-ufrj-bigdata-bi/blob/main/tcc_pedromaia_basesisu_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# Caminho para o arquivo na pasta do Google Drive
root_path = 'drive/MyDrive/TCC_UFRJ_BI/'

# Base 2022-2
sisu_2022_2 = 'chamada_regular_sisu_2022_2.csv'

# Base 2022-1
sisu_2022_1 = 'chamada_regular_sisu_2022_1.csv'

# Base 2021-2
sisu_2021_2 = 'chamada_regular_sisu_2021_2.csv'

# Base 2021-1
sisu_2021_1 = 'chamada_regular_sisu_2021_1.csv'

# Base 2020-2
sisu_2020_2 = 'chamada_regular_sisu_2020_2.csv'

# Base 2020-1
sisu_2020_1 = 'chamada_regular_sisu_2020_1.csv'

# Base 2019-2
sisu_2019_2 = 'chamada_regular_sisu_2019_2.csv'

# Base 2019-1
sisu_2019_1 = 'chamada_regular_sisu_2019_1.csv'

# Lista de colunas desejadas
colunas_desejadas = ['UF_IES', 'SIGLA_IES', 'NOME_CURSO', 'TIPO_MOD_CONCORRENCIA', 'NOTA_L', 'NOTA_CH', 'NOTA_CN', 'NOTA_M', 'NOTA_R', 'NOTA_CANDIDATO', 'APROVADO']
colunas_desejadas2 = ['SG_UF_IES', 'SIGLA_IES', 'NO_CURSO', 'TP_MOD_CONCORRENCIA', 'NU_NOTA_L', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_M', 'NU_NOTA_R', 'NU_NOTA_CANDIDATO', 'ST_APROVADO']

# Ler o arquivo CSV incluindo apenas as colunas desejadas
df_2022_2 = pd.read_csv(root_path + sisu_2022_2, sep='|', encoding='latin1', usecols=colunas_desejadas)
df_2022_1 = pd.read_csv(root_path + sisu_2022_1, sep='|', encoding='latin1', usecols=colunas_desejadas)
df_2021_2 = pd.read_csv(root_path + sisu_2021_2, sep='|', encoding='latin1', usecols=colunas_desejadas)
df_2021_1 = pd.read_csv(root_path + sisu_2021_1, sep=';', encoding='utf-8', usecols=colunas_desejadas2)
df_2020_2 = pd.read_csv(root_path + sisu_2020_2, sep=';', encoding='utf-8', usecols=colunas_desejadas2)
df_2020_1 = pd.read_csv(root_path + sisu_2020_1, sep=';', encoding='utf-8', usecols=colunas_desejadas2)
df_2019_2 = pd.read_csv(root_path + sisu_2019_2, sep=';', encoding='utf-8', usecols=colunas_desejadas2)
df_2019_1 = pd.read_csv(root_path + sisu_2019_1, sep=';', encoding='utf-8', usecols=colunas_desejadas2)

In [ ]:
# Lista de DataFrames
dataframes = [df_2022_2, df_2022_1, df_2021_2, df_2021_1, df_2020_2, df_2020_1, df_2019_2, df_2019_1]
#dataframes = [df_2021_1, df_2020_2, df_2020_1, df_2019_2, df_2019_1]
#dataframes = [df_2021_1, df_2020_2, df_2020_1, df_2019_2]
#dataframes = [df_2021_1, df_2020_2, df_2020_1]
#dataframes = [df_2021_1, df_2020_2]
#dataframes = [df_2021_2, df_2022_2]
#dataframes = [df_2022_2]

# Loop sobre os DataFrames
for df in dataframes:
    # Renomeando as colunas conforme as regras especificadas
    df.rename(columns={
        'SG_UF_IES': 'UF_IES',
        'SIGLA_IES': 'SIGLA_IES',
        'NO_CURSO': 'NOME_CURSO',
        'TP_MOD_CONCORRENCIA': 'TIPO_MOD_CONCORRENCIA',
        'NU_NOTA_L': 'NOTA_L',
        'NU_NOTA_CH': 'NOTA_CH',
        'NU_NOTA_CN': 'NOTA_CN',
        'NU_NOTA_M': 'NOTA_M',
        'NU_NOTA_R': 'NOTA_R',
        'NU_NOTA_CANDIDATO': 'NOTA_CANDIDATO',
        'ST_APROVADO': 'APROVADO'
    }, inplace=True)

In [ ]:
#Concatenando as Bases do SISU
df = pd.concat([df_2022_1, df_2022_2, df_2021_2, df_2021_1, df_2020_2, df_2020_1, df_2019_2, df_2019_1], ignore_index=True)
#df = pd.concat([df_2022_1, df_2022_2, df_2021_2, df_2021_1, df_2020_2, df_2020_1, df_2019_2], ignore_index=True)
#df = pd.concat([df_2022_1, df_2022_2, df_2021_2, df_2021_1, df_2020_2, df_2020_1], ignore_index=True)
#df = pd.concat([df_2022_1, df_2022_2, df_2021_2, df_2021_1, df_2020_2], ignore_index=True)
#df = pd.concat([df_2022_1, df_2022_2, df_2021_2, df_2021_1], ignore_index=True)
#df = pd.concat([df_2021_2, df_2022_2], ignore_index=True)
#df = pd.concat([df_2022_2, df_2021_2], ignore_index=True)
#df = pd.concat([df_2021_2, df_2022_2], ignore_index=True)
#df = pd.concat([df_2022_2], ignore_index=True)

In [ ]:
#removendo valores nulos
df.dropna(inplace = True)

# Substituir vírgulas por pontos e converter 'NOTA_CORTE' para tipo numérico
df['NOTA_L'] = df['NOTA_L'].str.replace(',', '.').astype(float)
df['NOTA_CH'] = df['NOTA_CH'].str.replace(',', '.').astype(float)
df['NOTA_CN'] = df['NOTA_CN'].str.replace(',', '.').astype(float)
df['NOTA_M'] = df['NOTA_M'].str.replace(',', '.').astype(float)
df['NOTA_CANDIDATO'] = df['NOTA_CANDIDATO'].str.replace(',', '.').astype(float)
df['NOME_CURSO'].replace(' ', '_', regex=True, inplace=True)
df['SIGLA_IES'].replace(' ', '_', regex=True, inplace=True)

# Transformando os resultados do campo APROVADO
df['APROVADO'] = df['APROVADO'].replace({'S': 1, 'N': 0})

# Mapeamento adicional para TIPO_MOD_CONCORRENCIA
df['TIPO_MOD_CONCORRENCIA'] = df['TIPO_MOD_CONCORRENCIA'].map({
    'L': 'C',
    'V': 'C',
    'B': 'C',
}).fillna(df['TIPO_MOD_CONCORRENCIA'])

df = df.rename(columns={
  #  'NOTA_CORTE': 'NOTACORTE',
    'NOTA_CANDIDATO': 'NOTACANDIDATO',
    'NOTA_L': 'NOTAL',
    'NOTA_CH': 'NOTACH',
    'NOTA_CN': 'NOTACN',
    'NOTA_M': 'NOTAM',
    'NOTA_R': 'NOTAR',
    'UF_IES': 'REGIAO',
    'SIGLA_IES': 'SIGLAIES',
    'NOME_CURSO': 'NOMECURSO',
    'TIPO_MOD_CONCORRENCIA': 'TIPOCONCORRENCIA'
})

In [ ]:
# Criar um dicionário de mapeamento para realizar o agrupamento
mapeamento_ufies = {
  'AC': 'NORTE',
  'AL': 'NORDESTE',
  'AP': 'NORTE',
  'AM': 'NORTE',
  'BA': 'NORDESTE',
  'CE': 'NORDESTE',
  'DF': 'CENTRO-OESTE',
  'ES': 'SUDESTE',
  'GO': 'CENTRO-OESTE',
  'MA': 'NORDESTE',
  'MT': 'CENTRO-OESTE',
  'MS': 'CENTRO-OESTE',
  'MG': 'SUDESTE',
  'PA': 'NORTE',
  'PB': 'NORDESTE',
  'PR': 'SUL',
  'PE': 'NORDESTE',
  'PI': 'NORDESTE',
  'RJ': 'SUDESTE',
  'RN': 'NORDESTE',
  'RS': 'SUL',
  'RO': 'NORTE',
  'RR': 'NORTE',
  'SC': 'SUL',
  'SP': 'SUDESTE',
  'SE': 'NORDESTE',
  'TO': 'NORTE'
  }

# Aplicar o mapeamento na coluna 'UFIES' para criar a nova coluna 'Região'
df['UFIES'] = df['REGIAO'].map(mapeamento_ufies)

In [ ]:
df.drop('REGIAO', axis=1, inplace=True)

In [ ]:
df['SIGLAIES']=df['SIGLAIES'].astype('category')
df['NOMECURSO']=df['NOMECURSO'].astype('category')
df['TIPOCONCORRENCIA']=df['TIPOCONCORRENCIA'].astype('category')
df['UFIES']=df['UFIES'].astype('category')

In [ ]:
df1 = df

In [ ]:
# Converter colunas categóricas em variáveis numéricas
df = pd.get_dummies(df)

In [ ]:
X = df.drop(['APROVADO'], axis=1).copy()

In [ ]:
y = df['APROVADO'].copy()

In [ ]:
from sklearn.model_selection import train_test_split

# Divisão dos dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [ ]:
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [ ]:
import xgboost as xgb
from statistics import mean

modelo_xgb = xgb.XGBClassifier()

In [ ]:
# Treinar o modelo final
modelo_xgb.fit(X_train, y_train)

# Previsões no conjunto de teste
predicoes_teste = modelo_xgb.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

# Métricas de avaliação
report = classification_report(y_test, predicoes_teste)

# Exibição do relatório
print(report)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Calcular a matriz de confusão
conf_matrix = confusion_matrix(y_test, predicoes_teste)

#Definir rótulos personalizados
class_names = ['Não Aprovado', 'Aprovado']

# Plotar a matriz de confusão com rótulos personalizados
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Previsão')
plt.ylabel('Real')
plt.title('Matriz de Confusão XGBClassifier')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Calcule as probabilidades de previsão
y_pred_prob = modelo_xgb.predict_proba(X_test)[:,1]

# Calcule as taxas de verdadeiros positivos e falsos positivos
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('Taxa de Falso Positivo')
plt.ylabel('Taxa de Verdadeiro Positivo')
plt.title('Curva ROC')
plt.show()

# Calcule a área sob a curva ROC (AUC)
print('AUC: ', roc_auc_score(y_test, y_pred_prob))

In [ ]:
bst = modelo_xgb.get_booster()
for importance_type in ('weight', 'gain', 'cover', 'total_gain', 'total_cover'):
  print('%s: ' % importance_type, bst.get_score(importance_type=importance_type))

  node_params = {'shape': 'box',
                 'style': 'filled, rounded',
                 'fillcolor': '#78cbe'}
  leaf_params = {'shape': 'box',
                 'style': 'filled',
                 'fillcolor': '#e48038'}

xgb.to_graphviz(modelo_xgb, num_trees=0, size="10,10",
                condition_node_params=node_params,
                leaf_node_params=leaf_params)

In [ ]:
################################## INPUT CANDIDATO ######################################################

In [ ]:
# Coletar as informações do candidato
TIPOCONCORRENCIA = input("Tipo de modalidade de concorrência: ").strip().upper()
REGIAO = input("Região do Brasil: ").strip().upper()
NOTACANDIDATO = float(input("Nota: "))

In [ ]:
# Filtrar o dataframe com base nas informações do candidato
df_filtrado = df1[(df1['TIPOCONCORRENCIA'] == TIPOCONCORRENCIA) &
                 (df1['UFIES'] == REGIAO) &
                 (df1['NOTACANDIDATO'] <= NOTACANDIDATO)]

# Exibir o dataframe filtrado
df_filtrado.head()

In [ ]:
# Converter colunas categóricas em variáveis numéricas
df_filtrado_encoded = pd.get_dummies(df_filtrado)

In [ ]:
# Separar a coluna alvo 'APROVADO'
X_filtrado = df_filtrado_encoded.drop(['APROVADO'], axis=1)
y_filtrado = df_filtrado_encoded['APROVADO']

In [ ]:
# Previsões usando o modelo XGBoost treinado
predicoes_aprovados = modelo_xgb.predict(X_filtrado)

In [ ]:
# Converter as previsões em um DataFrame com os índices redefinidos
predicoes_df = pd.DataFrame(predicoes_aprovados, columns=['PREVISAO_APROVADO'], index=X_filtrado.index)

# Concatenar os DataFrames redefinindo os índices
df_resultado = pd.concat([X_filtrado.reset_index(drop=True), predicoes_df.reset_index(drop=True)], axis=1)

In [ ]:
def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

df_resultado_reverse = undummify(X_filtrado)

df_resultado_reverse = df_resultado_reverse.reset_index(drop=True)

In [ ]:
df_resultado_reverse.head()

In [ ]:
pred_aprovados = pd.DataFrame(predicoes_aprovados)
pred_aprovados = pred_aprovados.rename(columns={0: "PREVISAO_APROVADO"})

df_final = pd.concat([df_resultado_reverse, pred_aprovados], axis=1)

df_final.head()

In [ ]:
# Filtrar apenas os resultados iguais a 1 na coluna 'PREVISAO_APROVADO'
df_aprovados = df_final[df_final['PREVISAO_APROVADO'] == 1]

# Eliminar as colunas e remover os underscores
df_aprovados = df_aprovados.drop(['PREVISAO_APROVADO', 'NOTAL', 'NOTACH', 'NOTACN', 'NOTAM', 'NOTAR'], axis=1)
df_aprovados['SIGLAIES'] = df_aprovados['SIGLAIES'].str.replace('_', ' ')
df_aprovados['NOMECURSO'] = df_aprovados['NOMECURSO'].str.replace('_', ' ')

In [ ]:
# Criar uma nova coluna com o valor mínimo de NOTACANDIDATO por SIGLAIES, NOMECURSO, TIPOCONCORRENCIA e UFIES
df_aprovados['NOTA'] = df_aprovados.groupby(['SIGLAIES', 'NOMECURSO', 'TIPOCONCORRENCIA', 'UFIES'])['NOTACANDIDATO'].transform('min')

# Eliminar o campo 'NOTACANDIDATO'
df_aprovados = df_aprovados.drop(['NOTACANDIDATO'], axis=1)

# Remover duplicatas
df_aprovados = df_aprovados.drop_duplicates()

# Ordenar df_aprovados pelo campo MIN_NOTACANDIDATO em ordem decrescente
df_aprovados = df_aprovados.sort_values(by='NOTA', ascending=False)

In [ ]:
# Verificar se df_resultado está vazio
if df_aprovados.empty:
    mensagem = "Que pena! Com sua nota e preferências, o modelo não conseguiu prever nenhuma aprovação."
else:
    mensagem = f"Parabéns com Nota {NOTACANDIDATO} no SISU, Modalidade de Concorrência {TIPOCONCORRENCIA} e região {REGIAO}, o modelo prevê que você conseguiria aprovação nos seguintes cursos e instituições:\n\n"
    for index, row in df_aprovados.iterrows():
        mensagem += f"\n{row['NOMECURSO']} em {row['SIGLAIES']} onde a nota de corte mínima histórica foi de {row['NOTA']}. \n"

# Exibir a mensagem
print(mensagem)